In [113]:
string = 'hej med dig. Har du det godt? Nej!'
string2 = '\nNEJ!'
print(string+string2)
string.split()


csv_string = '''var1,var2,var3,var4,var5,var6\n1,2,3,4,5,6,10'''
for line in csv_string.split('\n'):
    print(line.split(','))

hej med dig. Har du det godt? Nej!
NEJ!
['var1', 'var2', 'var3', 'var4', 'var5', 'var6']
['1', '2', '3', '4', '5', '6', '10']


In [122]:
liste = range(5)
liste[3]
count = 0
for i in liste:
    count+=i
    print(count,i)



0 0
1 1
3 2
6 3
10 4


In [136]:
dictionary = {'snorre':{'seje_ting':['Hallo jeg er nummer 1','Han er sje'],'venner':['hans','pater']},'sociologi':['']}
dictionary['snorre']['venner'][0].split('a')[0].upper()

'H'

In [139]:
f = open('test.csv','w')
f.write('hej igen.')
f.close()

**<center>Accessing data from the web</center>**
***<center>Crawling, scraping, and APIs</center>***

<center>Snorre Ralund</center>

** Todays message **
* Utilize the datasources around you. (Job data and crime)
* Knowing how to create your own custom datasets pulling information from many different sources.
* You should know all the tricks, but use them with care. 

**Agenda**

* The basics of webscraping
    * Connecting, Crawling, Parsing, Storing, Logging.
* Hacks: Backdoors, url construction, and analysis of a webpage.
* Reliability of your datacollection.
* Screen-scraping - Automated browsing
    * Interactions:
        * Login in, scrolling, pressing buttons.
* APIs 
    * Authentication
    * Building queries

## Ethics / Legal Issues
* If a regular user can’t access it, we shouldn’t try to get it (That is considered hacking)https://www.dr.dk/nyheder/penge/gjorde-opmaerksom-paa-cpr-hul-nu-bliver-han-politianmeldt-hacking. 
* Don't hit it to fast: Essentially a DENIAL OF SERVICE attack (DOS). [Again considered hacking](https://www.dr.dk/nyheder/indland/folketingets-hjemmeside-ramt-af-hacker-angreb). 
* Add headers stating your name and email with your requests to ensure transparency. 
* Be careful with copyrighted material.
* Fair use (don't take everything)
* If monetizing on the data, be careful not to be in direct competition with whom you are taking the data from.

<img src="https://github.com/snorreralund/images/raw/master/Sk%C3%A6rmbillede%202017-08-03%2014.46.32.png"/>

## Setting up the essentials:
Good practices:
* Transparency
* Ratelimiting
* Reliability

In [150]:
# Transparent scraping
import requests
url = 'https://www.facebook.com'
#response = requests.get(url) # url, address of the site and instructions on where you wanna go.
session = requests.session()
#session.headers['email'] = 'youremail' 
#session.headers['name'] = 'name'
session.headers # Who you are, what format you want, and authentification.
#response.text
response = session.get(url)

A quick tip is that you can change the user agent to a cellphone to obtain more simple formatting of the html. 

In [151]:
# Control the pace of your calls
import time
def ratelimit():
    "A function that handles the rate of your calls."
    time.sleep(1)
# Reliable requests
def get(url,iterations=10,exceptions=(Exception)):
    """This module ensures that your script does not crash from connection errors.
        iterations : Define number of iterations before giving up. 
        exceptions: Define which exceptions you accept, default is all. 
    """
    for iteration in range(iterations):
        try:
            # add ratelimit function call here
            ratelimit() # !!
            response = session.get(url)
            return response # if succesful it will end the iterations here
        except exceptions as e: #  find exceptions in the request library requests.exceptions
            print(e) # print or log the exception message.
    return None # your code will purposely crash if you don't create a check function later.

In [152]:
# Interactive browsing - more on this later / google geckodriver
from selenium import webdriver
path2gecko = '/Users/axelengbergpallesen/Downloads/geckodriver' # define path to your geckodriver
browser = webdriver.Firefox(executable_path=path2gecko)
browser.get('https://www.facebook.com')

In [158]:
sel = '#u_0_2'
element = browser.find_element_by_css_selector(sel)

#username = 'snorreralund@hotmail.com'
#element.send_keys(username)
element.click()

In [ ]:
response = get() 

** HTML is a mess.** 

For now lets look at how to collect well-structured data.

# APIS
For fast, efficient and ***reliable*** data collection.

Only catch is that they control the amounts, and which endpoints you can collect 

also they **change**.
    - e.g. facebook cancelled 
        - querying friendship relations (without having users signing up to your app), 
        - group activity without admin rights, 
        - and most recently the ability to trace public activity (likes and comments) without admin rights.
    - twitter (and more recently facebook) will not let you collect all historic activity --> streaming data.

Begins with reading the docs... 
- getting authentification - creating apps, getting and renewing tokens - 
- building queries.
- ratelimiting and pagination.

Often comes in the Json format. --> nested dictionaries and lists.

Example: Explore the facebook api here: https://developers.facebook.com/tools/explorer/

In [163]:
token = 'EAACEdEose0cBAC4X6wT0DLfpwXVKBH5X0VQyXoRS5CxLSxTjtnEZCIK4WaeS1cvDDkKxfx7asQdBgNEy2ni71SVoqTwSqnZAJNByCzOfVhRD8GDIodon8RmN9cDyZCatRFImldFqT8PPDP6sGtgcVspEsPGvnZBldZBkulzDGkdZCoyZBHS7rKtQ4Gjx5DqJTYZD'
page_id = '62507427296'
q = '%s?fields=likes'%page_id

base_url = 'https://graph.facebook.com/v2.12/'
authentication = '&access_token=%s'%token
url = base_url + q + authentication


response = get(url)

In [167]:
import json

response_json = json.loads(response.text)

In [178]:
ids = []
for page in response_json['likes']['data']:
    ids.append(page['id'])


In [ ]:

for page_id in ids:
    q = '%s?fields=likes'%page_id

    base_url = 'https://graph.facebook.com/v2.12/'
    authentication = '&access_token=%s'%token
    url = base_url + q + authentication
    

## APIS: Collect data from Twitter

In [17]:
# XXX: Go to http://dev.twitter.com/apps/new to create an app and get values
# for these credentials that you'll need to provide in place of these
# empty string values that are defined as placeholders.
# See https://developer.twitter.com/en/docs/basics/authentication/overview/oauth for more information 
# on Twitter's OAuth implementation

CONSUMER_KEY=""
CONSUMER_SECRET=""
OAUTH_TOKEN=""
OAUTH_TOKEN_SECRET=""

In [179]:
import pickle
#pickle.dump([CONSUMER_KEY,CONSUMER_SECRET,OAUTH_TOKEN,OAUTH_TOKEN_SECRET],open('twitter_credentials.pkl','wb'))
CONSUMER_KEY,CONSUMER_SECRET,OAUTH_TOKEN,OAUTH_TOKEN_SECRET = pickle.load(open('twitter_credentials.pkl','rb'))

In [221]:
# How to use this authentification?
# answer from https://stackoverflow.com/questions/33308634/how-to-perform-oauth-when-doing-twitter-scrapping-with-python-requests
from requests_oauthlib import OAuth1

url ='https://api.twitter.com/1.1/search/tweets.json?q=foucault'#&geocode=55.676098,12.568337,5km' 
auth = OAuth1(CONSUMER_KEY, CONSUMER_SECRET, OAUTH_TOKEN, OAUTH_TOKEN_SECRET)
response = requests.get(url, auth=auth)

In [222]:
response_json = response.json()

In [223]:
import pandas as pd
ststus_df = pd.DataFrame(response_json['statuses'])
#response_json['statuses'][3]['text']#.keys()

In [224]:
users = []
for status in response_json['statuses']:
    user = status['user']
    users.append(user)

    #status_df['user']#.values[0]

In [225]:
df_users = pd.DataFrame(users)

In [241]:
#! mkdir twitter_data
! mkdir twitter_data/foucault_data

In [245]:
path = 'twitter_data/foucault_data/'

count = 0
while True:
    base_q = 'https://api.twitter.com/1.1/search/tweets.json'
    next_link = response_json['search_metadata']['next_results']
    url = base_q+next_link
    response = get(url)
    if response.ok:
        response_json = response.json()
    
    filename = path + 'foucault_%d'%count
    f = open(filename,'w')
    json.dump(response_json,f)
    count+=1
    if count>10:
        break

In [238]:
response = get('https://api.twitter.com/1.1/search/tweets.json'+response_json['search_metadata']['next_results'])##['statuses']

{'search_metadata': {'completed_in': 0.039,
  'count': 15,
  'max_id': 964532712437624831,
  'max_id_str': '964532712437624831',
  'next_results': '?max_id=964529302736957441&q=foucault&include_entities=1',
  'query': 'foucault',
  'refresh_url': '?since_id=964532712437624831&q=foucault&include_entities=1',
  'since_id': 0,
  'since_id_str': '0'},
 'statuses': [{'contributors': None,
   'coordinates': None,
   'created_at': 'Fri Feb 16 16:10:11 +0000 2018',
   'entities': {'hashtags': [],
    'media': [{'display_url': 'pic.twitter.com/hhpoUxGvnG',
      'expanded_url': 'https://twitter.com/rionamaTeL/status/964532368139792389/photo/1',
      'id': 964531767708454912,
      'id_str': '964531767708454912',
      'indices': [97, 120],
      'media_url': 'http://pbs.twimg.com/media/DWK0iEzXkAA_Bln.jpg',
      'media_url_https': 'https://pbs.twimg.com/media/DWK0iEzXkAA_Bln.jpg',
      'sizes': {'large': {'h': 1197, 'resize': 'fit', 'w': 1505},
       'medium': {'h': 954, 'resize': 'fit', 'w

In [213]:
# See the ratelimits by looking at the response headers
wait_time = time.time()
time.sleep(wait_time)
response.headers

{'cache-control': 'no-cache, no-store, must-revalidate, pre-check=0, post-check=0', 'content-disposition': 'attachment; filename=json.json', 'content-encoding': 'gzip', 'content-length': '2290', 'content-type': 'application/json;charset=utf-8', 'date': 'Fri, 16 Feb 2018 16:10:06 GMT', 'expires': 'Tue, 31 Mar 1981 05:00:00 GMT', 'last-modified': 'Fri, 16 Feb 2018 16:10:06 GMT', 'pragma': 'no-cache', 'server': 'tsa_o', 'set-cookie': 'personalization_id="v1_nPHoYqGvPl/LCsRNZR3sOA=="; Expires=Sun, 16 Feb 2020 16:10:06 UTC; Path=/; Domain=.twitter.com, lang=da; Path=/, guest_id=v1%3A151879740625586081; Expires=Sun, 16 Feb 2020 16:10:06 UTC; Path=/; Domain=.twitter.com', 'status': '200 OK', 'strict-transport-security': 'max-age=631138519', 'x-access-level': 'read', 'x-connection-hash': '8c461a5281816b64ed1f3aa383555d07', 'x-content-type-options': 'nosniff', 'x-frame-options': 'SAMEORIGIN', 'x-rate-limit-limit': '180', 'x-rate-limit-remaining': '179', 'x-rate-limit-reset': '1518798306', 'x-re

In [215]:
# We want to include the authentification in all calls to twitter
session.auth = auth

In [89]:
response_json = response.json()

In [84]:
# Now lets inspect the response
import pandas as pd
status_df = pd.DataFrame(response_json['statuses']) 
#response_json['statuses'][4]

# Case: Collecting data about danish politics 

## searching for hashtags

In [216]:
#url ='https://api.twitter.com/1.1/search/tweets.json?q=dkpol' 
#response_json['statuses'][4].keys()

In [91]:
#response_json['statuses'][12]['retweeted_status']#['retweet_count']


### paging results

In [92]:
#! mkdir  # create directory for the data

In [ ]:
base_path = ''
for i in range(5): # get the next 5 results
    next_link = response_json['search_metadata']['next_results'] # grab paging link
    max_id = response_json['search_metadata']['max_id']
    # make another call
    response = get(next_link)
    # dump the raw data
    filename = base_path+'search_%s_%d'%(search_query,max_id)
    f = open(filename,'w')
    f.write(response.text)
    f.close()

    

## getting retweets

In [74]:
tweet_id = response_json['statuses'][12]['retweeted_status']['id']

In [79]:

# get retweets
url = 'https://api.twitter.com/1.1/statuses/retweets/%d.json'%tweet_id
response = get(url)

In [95]:
# dump the data as network data.

## Exercise: Monitoring danish politicians on twitter
A twitter-user with the twitter handle *politikere* has been so kind to curate a list of danish politicians.

**1)** Figure out how to construct an API call to collect who this user follows. (Look in the API reference index)

**2)** Next you should construct an API call to retrieve the statuses of those politicians.

**extra** 
- Make a loop to collect who each politician follows (beware of ratelimits).
- Construct a network of politicians using the Networkx package 